In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np
import pandas as pd
import datetime

import src.data_cleaning as dc

In [4]:
import matplotlib as plt

ModuleNotFoundError: No module named 'matplotlib'

In [3]:
# defining the 4 dataframes from our .py file

df_sales, df_resbldg, df_parcel, df_lookup = dc.create_dfs('EXTR_RPSale.csv', 
                                                           'EXTR_ResBldg.csv', 
                                                           'EXTR_Parcel.csv',
                                                           'EXTR_LookUp.csv')

/Users/kyledecember1/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/kyledecember1/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
# filter for sales from 2019

df_sales_2019 = dc.sales_2019(df_sales)

/Users/kyledecember1/mod_2/project/king_county_housing_analysis/src/data_cleaning.py:32: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  df = df[df['datetime'] > yr2018]
/Users/kyledecember1/mod_2/project/king_county_housing_analysis/src/data_cleaning.py:33: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  df = df[df['datetime'] < yr2020]


In [5]:
# combine major + minor to create HID column

df_sales_2019['HID'] = (df_sales_2019['Major'].astype(str)).str.zfill(6) + '-' + (df_sales_2019['Minor'].astype(str)).str.zfill(4)
df_resbldg['HID'] = (df_resbldg['Major'].astype(str)).str.zfill(6) + '-' + (df_resbldg['Minor'].astype(str)).str.zfill(4)
df_parcel['HID'] = (df_parcel['Major'].astype(str)).str.zfill(6) + '-' + (df_parcel['Minor'].astype(str)).str.zfill(4)

In [6]:
#retain only relevant columns for df_sales_2019

columns = ['Major', 'Minor', 'DocumentDate', 'SalePrice', 'PropertyType', 'HID']
df_sales_2019 = df_sales_2019.loc[:, columns]

In [ ]:
# new column for whether or not the location is waterfront property
df_parcel['waterfront'] = df_parcel['WfntLocation'] > 0

In [8]:
df_parcel['AirportNoise'].unique()
print (type(df_parce['AirportNoise'])

array([ 0, 67, 59, 60, 66, 63, 61, 64, 65, 70, 62, 69, 68,  1, 50])

In [9]:
df_parcel['TrafficNoise'].unique()

array([0, 1, 2, 3])

In [11]:
df_lookup[df_lookup['LUType'] == 50]

,LUType,LUItem,LUDescription
964,50,1,DUWAMISH ...
965,50,2,ELLIOTT BAY ...
966,50,3,PUGET SOUND ...
967,50,4,LAKE UNION ...
968,50,5,SHIP CANAL ...
969,50,6,LAKE WASH ...
970,50,7,LAKE SAMM ...
971,50,8,OTHER LAKE ...
972,50,9,RIVER/SLOUGH ...
